# (저와 같은) 초보 중에서도 초보를 위한 베이스라인

### 저와 같은 초보 중에서도 초보들이 저처럼 시간낭비?를 하지 않기를 바라는 마음으로 이렇게 코드공유를 해봅니다.

####! 이미 한번이라도 제출해본 적이 있으신 분은 안보고 넘어가셔도 좋습니다 !



이 코드 공유는 아래의 어려움을 겪으신 분들을 위한 베이스라인? 코드입니다.
- 데이콘이라는걸 처음 알게 된 분들이 **경험삼아 한번이라도 제출하고 싶은 경우**
- 베이스라인대로 코드를 돌려봤는데 데이터프레임 칼럼명(헤더)가 깨져서 당황하신 분들
- 제공되는 기본 베이스라인코드를 그대로 돌려봤는데 본인이 코드 실행해서 만들어진 **submission.csv와 sample_submission.csv가 서로 다르게 생겨서** 당황하신 분들
- sklearn에서 **GridSearchCV**를 써본적이 없는 분들
- **SMAPE**를 어떻게 loss함수로 지정해야하는지 몰라 포기하셨던 분들



거의 대다수의 분들은 위의 사항에 해당되지 않음을 알고있습니다...만 그래도 저같이 판다스 람다함수 같은 기본적인 것도 못 외워서 검색하고 어디를 실수했는지 못찾아서 거의 하루를 매달려야 하는 분들도 있기는 있을 것이라고 생각하여 올려봅니다


(주의) 기존 베이스라인코드를 많이 베껴왔습니다.

# 데이터 전처리

In [134]:
#Library Imports
import numpy as np  # 넘파이
import pandas as pd  # 판다스
# import math
import os  # 디렉토리 변경
from sklearn.model_selection import GridSearchCV  # 파라미터 설정 고민을 줄여주는 고마운 친구
from sklearn.metrics import make_scorer  # loss function 커스터마이징

from lightgbm import LGBMRegressor  # 전 LGBMRegressor 만을 사용해서 돌려보았습니다.
from sklearn.model_selection import train_test_split  # train, valid set 제작

import warnings
warnings.filterwarnings(action='ignore')


#os.chdir('/content/drive/MyDrive/Colab Notebooks/데이콘/전력사용량 예측') 

In [117]:
# 데이터 로드 (인코딩은 euc-kr)
train=pd.read_csv('train.csv', encoding='euc-kr')
test=pd.read_csv('test.csv', encoding='euc-kr')
submission=pd.read_csv('sample_submission.csv', encoding='euc-kr')

In [5]:
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0


In [6]:
test  # 전력사용량 없음

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1.0,1.0
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
#건물별로 '비전기냉방설비운영'과 '태양광보유'를 판단해 test set의 결측치를 보간해줍니다
# 건물별로의 비전기냉방설비운영, 태양광보유는 같은 건물로 위와 같으므로, 위에 값으로 대체해 줄 수 있다.
train[['num', '비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
# 60개의 건물로 60개로 나누어준다.
for i in range(0, len(train), len(train)//60):
    count +=1
    ice[count]=train.loc[i,'비전기냉방설비운영']
    hot[count]=train.loc[i,'태양광보유']

for i in range(len(test)):
    test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
    test.loc[i, '태양광보유']=hot[test['num'][i]]

In [119]:
# 시간, 요일, 주말여부(new!) 추가
def time(x):
    return int(x[-2:])
train['time']=train['date_time'].apply(lambda x: time(x))
test['time']=test['date_time'].apply(lambda x: time(x))

# 평일=0~4, 주말=5~6
def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))

# 평일=0, 주말=1
train['weekend']=train['weekday'].apply(lambda x: 0 if x < 4 else 1)
test['weekend']=test['weekday'].apply(lambda x: 0 if x < 4 else 1)

In [120]:
# 기온, 풍속, 습도 등, 기타 결측치를 적당히 1/3, 2/3 수치로 보간해줍니다.
test = test.interpolate(method='values')  
test.head(3)

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,time,weekday,weekend
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.0,0.0,0.0,0,1,0
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.0,0.0,0.0,1,1,0
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.0,0.0,0.0,2,1,0


In [10]:
train.head(3)

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,time,weekday,weekend
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,0,0,0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,1,0,0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,2,0,0


# 모델링

In [121]:
# 학습용set 생성
train.drop('date_time', axis=1, inplace=True)  # 학습에 불필요한 날짜 제거
train_x=train.drop('전력사용량(kWh)', axis=1)  # 문제
train_y=train[['전력사용량(kWh)']]  # 정답

# X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=156)

In [122]:
df_x_53 = train_x[train_x.num == 53]
df_y_53 = train_y[train_x.num == 53]
x_train_53 = df_x_53.iloc[:-72,]
x_test_53 = df_x_53.iloc[-72:,]
y_train_53 = df_y_53.iloc[:-72,]
y_test_53 = df_y_53.iloc[-72:,]

#X_train, X_valid, y_train, y_valid = train_test_split(x_train_53, y_train_53, test_size=0.2, random_state=156)

In [13]:
df_x_53

,num,기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,time,weekday,weekend
106080,53,19.9,1.4,83.0,0.0,0.0,1.0,0.0,0,0,0
106081,53,19.3,0.9,89.0,0.0,0.0,1.0,0.0,1,0,0
106082,53,18.5,0.7,94.0,0.2,0.0,1.0,0.0,2,0,0
106083,53,18.5,0.3,96.0,0.4,0.0,1.0,0.0,3,0,0
106084,53,18.6,1.5,95.0,0.0,0.0,1.0,0.0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...
108115,53,31.1,1.1,67.0,0.0,0.6,1.0,0.0,19,0,0
108116,53,29.3,0.7,80.0,0.0,0.0,1.0,0.0,20,0,0
108117,53,28.6,0.9,84.0,0.0,0.0,1.0,0.0,21,0,0
108118,53,28.3,1.3,88.0,0.0,0.0,1.0,0.0,22,0,0


In [14]:
df_y_53

,전력사용량(kWh)
106080,958.554
106081,924.210
106082,920.484
106083,921.618
106084,916.110
...,...
108115,1498.014
108116,1445.202
108117,1359.504
108118,1347.840


In [15]:
x_train_53.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1968 entries, 106080 to 108047
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   num        1968 non-null   int64  
 1   기온(°C)     1968 non-null   float64
 2   풍속(m/s)    1968 non-null   float64
 3   습도(%)      1968 non-null   float64
 4   강수량(mm)    1968 non-null   float64
 5   일조(hr)     1968 non-null   float64
 6   비전기냉방설비운영  1968 non-null   float64
 7   태양광보유      1968 non-null   float64
 8   time       1968 non-null   int64  
 9   weekday    1968 non-null   int64  
 10  weekend    1968 non-null   int64  
dtypes: float64(7), int64(4)
memory usage: 184.5 KB


In [124]:
import os
import glob
def smape(true, pred):

    v = 2 * abs(pred - true) / (abs(pred) + abs(true))

    output = np.mean(v) * 100
    eval_name = 'smape'
    is_higher_better = False
    
    return (eval_name, output, is_higher_better)

In [50]:
# 파라미터 설정, 모델생성 함수
# def get_best_params(model, params):
#     grid_model = GridSearchCV(
#         model,
#         param_grid = params,  # 파라미터
#         cv=5,  # Kfold : 5
#         scoring= smape)  #loss function
    
#     grid_model.fit(x_train_53, y_train_53, eval_set=(x_test_53, y_test_53))
#     scr = grid_model.best_score_
#     print(f'{model.__class__.__name__} 최적 score 값 {scr}')
#     return grid_model.best_estimator_
#     grid_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=100)
#     scr = grid_model.best_score_
#     print(f'{model.__class__.__name__} 최적 score 값 {scr}')
#     return grid_model.best_estimator_

In [51]:
# 파라미터 후보군 설정
# 어떤 파라미터로 하는게 좋을지 고민된다면 고민하는 것들을 리스트 안에 다 넣어보세요 알아서 골라줄겁니다.
# 저는 예시로 learning_rate만 0.1 or 0.01 중 더 좋은걸 골라달라고 했습니다.
# params = {}
# params['boosting_type'] = ['gbdt']
# params['objective'] = ['regression']
# params['n_estimators'] = [100]
# params['learning_rate'] = [0.1, 0.01]  
# params['subsample'] = [1]

In [125]:
# 모델정의
model=LGBMRegressor(n_estimators=200, objective='rmse', random_state=156)
model.fit(x_train_53, y_train_53, eval_set=(x_test_53, y_test_53), 
            early_stopping_rounds=30, verbose=100, eval_metric = ['mse', 'rmse', 'mape', smape])

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[28]	valid_0's l2: 51352	valid_0's mape: 0.0625141	valid_0's rmse: 226.61	valid_0's smape: 6.79792


LGBMRegressor(n_estimators=200, objective='rmse', random_state=156)

In [53]:
# # 학습진행
# best_lgbm = get_best_params(model, params)
# best_lgbm  # learning_rate 0.1 or 0.01 중, 0.01이 더 좋았다고 하는군요. 

In [126]:
# 기타 지표로 에러 측정
from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(x_test_53)

mse_score = mean_squared_error(y_test_53, y_pred)
r2_score = r2_score(y_test_53, y_pred)
print('MSE:', mse_score)
print('R2 :', r2_score)

MSE: 51352.02262658086
R2 : 0.7425572921357801


In [ ]:
lgbm_re = LGBMRegressor(n_estimators=200)

params = {'boosting_type' : ['gbdt'],
          'objective' : ['regression'],
         'n_estimators' : [150, 200, 250, 300, 400],
         'learning_rate' : [0.5, 0.1, 0.05, 0.01],
          'subsample' : [0.5, 0.8, 1],
          'num_leaves' : [10, 20, 30]
          #'min_child_samples' : [1, 5, 10, 15, 20],
          #'max_depth' : [0.1, 0.5, 0.01, -1]
         }

gridcv = GridSearchCV(lgbm_re, param_grid=params, cv=5)
gridcv.fit(x_train_53, y_train_53, early_stopping_rounds=30, eval_set=(x_test_53, y_test_53), eval_metric=smape)

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)

scr = gridcv.best_score_
print('최적 score 값', scr)


# params['boosting_type'] = ['gbdt']
# params['objective'] = ['regression']
# params['n_estimators'] = [100]
# params['learning_rate'] = [0.1, 0.01]  
# params['subsample'] = [1]

In [140]:
model=LGBMRegressor(n_estimators=150, learning_rate=0.1, boosting_type='gbdt', num_leaves=10, objective='regression', subsample=0.5,
                   random_state=156)
model.fit(x_train_53, y_train_53, eval_set=(x_test_53, y_test_53), 
            early_stopping_rounds=30, verbose=100, eval_metric = ['mse', 'rmse', 'mape', smape])

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[23]	valid_0's l2: 46665.1	valid_0's rmse: 216.021	valid_0's mape: 0.0656831	valid_0's smape: 7.03486


LGBMRegressor(n_estimators=150, num_leaves=10, objective='regression',
              random_state=156, subsample=0.5)

In [141]:
y_pred = model.predict(x_test_53)
from sklearn.metrics import r2_score
mse_score = mean_squared_error(y_test_53, y_pred)
r2_score = r2_score(y_test_53, y_pred)
print('MSE:', mse_score)
print('R2 :', r2_score)

MSE: 46665.068740854884
R2 : 0.7660543627137062


In [246]:
# # 모델저장, 로드
# from sklearn.externals import joblib
# joblib.dump(best_lgbm, 'best_lgbm.pkl')
# load_lgbm = joblib.load('best_lgbm.pkl')

# 제출용 데이터

In [19]:
# # 모델에 넣기 위해 날짜칼럼 제거
# test_x = test.drop('date_time', axis=1)

In [20]:
# # 모델 예측
# submission_y = best_lgbm.predict(test_x)

In [21]:
# # submission.csv 생성
# test['answer'] = submission_y
# test['num_date_time'] = test.apply(lambda x: str(x['num']) +' '+ x['date_time'], axis=1)
# submission = test[['num_date_time', 'answer']]
# submission.to_csv('submission.csv', index=False)
# submission

,num_date_time,answer
0,1 2020-08-25 00,6264.865912
1,1 2020-08-25 01,6264.865912
2,1 2020-08-25 02,6264.865912
3,1 2020-08-25 03,6264.865912
4,1 2020-08-25 04,6264.865912
...,...,...
10075,60 2020-08-31 19,2809.636990
10076,60 2020-08-31 20,2392.383243
10077,60 2020-08-31 21,2343.381185
10078,60 2020-08-31 22,2343.381185
